# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [3]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


In [4]:
# load messages dataset
messages = pd.read_csv("disaster_messages.csv", header=0).set_index("id")
messages.head()

,message,original,genre
id,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [5]:
# load categories dataset
categories = pd.read_csv("disaster_categories.csv", header=0).set_index("id")
categories.head()

,categories
id,
2,related-1;request-0;offer-0;aid_related-0;medi...
7,related-1;request-0;offer-0;aid_related-1;medi...
8,related-1;request-0;offer-0;aid_related-0;medi...
9,related-1;request-1;offer-0;aid_related-1;medi...
12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [6]:
# merge datasets
df = messages.join(categories, how="inner")
df.head()

,message,original,genre,categories
id,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [7]:
# create a dataframe of the 36 individual category columns
categories = df.categories.str.split(";", expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
id,,,,,,,,,,,,,,,,,,,,,
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
7,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
8,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
9,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
12,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [8]:
# select the first row of the categories dataframe
row = categories.iloc[0, :]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.str.extract(r"([a-z\_]*)")[0].tolist()
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [9]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
7,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
8,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
9,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
12,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [10]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str[-1]
    
    # convert column from string to numeric
    categories[column] = pd.to_numeric(categories[column]).replace(2, 1)
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
categories[categories.related==3]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [12]:
# drop the original categories column from `df`
df = pd.concat([df.drop("categories", axis=1), categories], axis=1)

df.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [13]:
# check number of duplicates
df.duplicated().sum()


172

In [14]:
# drop duplicates
df = df.drop_duplicates()

In [15]:
# check number of duplicates
df.duplicated().sum()


0

### 7. Explore data
- categories distribution


In [26]:
categories_names = df.columns[4:]
categories_names

Index(['request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [31]:
categories_counts = df[categories_names].sum().sort_values(ascending=False)
categories_counts

aid_related               10860
weather_related            7297
direct_report              5075
request                    4474
other_aid                  3446
food                       2923
earthquake                 2455
storm                      2443
shelter                    2314
floods                     2155
medical_help               2084
infrastructure_related     1705
water                      1672
other_weather              1376
buildings                  1333
medical_products           1313
transport                  1201
death                      1194
other_infrastructure       1151
refugees                    875
military                    860
search_and_rescue           724
money                       604
electricity                 532
cold                        530
security                    471
clothing                    405
aid_centers                 309
missing_people              298
hospitals                   283
fire                        282
tools   

In [29]:
genre_counts = df.groupby('genre').count()['message']
#genre_names = list(genre_counts.index)
genre_counts

genre
direct    10766
news      13053
social     2395
Name: message, dtype: int64

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26214 entries, 2 to 30265
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   message                 26214 non-null  object
 1   original                10170 non-null  object
 2   genre                   26214 non-null  object
 3   related                 26214 non-null  int64 
 4   request                 26214 non-null  int64 
 5   offer                   26214 non-null  int64 
 6   aid_related             26214 non-null  int64 
 7   medical_help            26214 non-null  int64 
 8   medical_products        26214 non-null  int64 
 9   search_and_rescue       26214 non-null  int64 
 10  security                26214 non-null  int64 
 11  military                26214 non-null  int64 
 12  child_alone             26214 non-null  int64 
 13  water                   26214 non-null  int64 
 14  food                    26214 non-null  int64 
 15  sh

In [39]:
list(categories.columns)

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [75]:
pd.melt(categories, value_vars=list(categories.columns))

,variable,value
0,related,1
1,related,1
2,related,1
3,related,1
4,related,1
...,...,...
949891,direct_report,0
949892,direct_report,0
949893,direct_report,0
949894,direct_report,0


In [50]:
pd.melt(categories,)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30263,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
categories

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30263,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
combination_occurences = (categories.reset_index()
    .groupby(list(categories.columns))
    .agg({"id":"count"}).rename(columns={'id':'occurences'})
    .sort_values("occurences", ascending=False)
    .reset_index().reset_index().rename(columns={"index": "id"}))

combination_occurences

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,occurences
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6140
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5345
2,2,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,725
3,3,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,441
4,4,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3997,3997,1,0,0,1,0,0,1,0,0,...,0,1,1,0,0,1,0,1,0,1
3998,3998,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3999,3999,1,0,0,1,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,1
4000,4000,1,0,0,1,0,0,1,0,0,...,0,1,1,1,0,0,0,0,1,1


In [92]:
combination_occurences_top20 = combination_occurences.iloc[:20]
combination_occurences_top20

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,occurences
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6140
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5345
2,2,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,725
3,3,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,441
4,4,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,433
5,5,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,361
6,6,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,303
7,7,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,284
8,8,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,253
9,9,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,250


In [90]:
combination_occurences_melt = pd.melt(combination_occurences, id_vars=["id", "occurences"], value_vars=list(categories.columns))
combination_occurences_melt

,id,occurences,variable,value
0,0,6140,related,0
1,1,5345,related,1
2,2,725,related,1
3,3,441,related,1
4,4,433,related,1
...,...,...,...,...
144067,3997,1,direct_report,0
144068,3998,1,direct_report,0
144069,3999,1,direct_report,0
144070,4000,1,direct_report,1


In [91]:
combination_occurences_melt.loc[combination_occurences_melt.value == 1].groupby("id").agg({"variable": list, "occurences":"first"})#.join(combination_occurences["id"])

,variable,occurences
id,,
1,[related],5345
2,"[related, weather_related, earthquake]",725
3,"[related, weather_related, storm]",441
4,"[related, request, aid_related, other_aid, dir...",433
5,"[related, request, aid_related, food, direct_r...",361
...,...,...
3997,"[related, aid_related, search_and_rescue, othe...",1
3998,"[related, aid_related, search_and_rescue, othe...",1
3999,"[related, aid_related, search_and_rescue, othe...",1


In [43]:
df.groupby(list(categories.columns)).count().head()


message  \
related request offer aid_related medical_help medical_products search_and_rescue security military child_alone water food shelter clothing money missing_people refugees death other_aid infrastructure_related transport buildings electricity tools hospitals shops aid_centers other_infrastructure weather_related floods storm fire earthquake cold other_weather direct_report            
0       0       0     0           0            0                0                 0        0        0           0     0    0       0        0     0              0        0     0         0                      0         0         0           0     0         0     0           0                    0               0      0     0    0          0    0             0                 6121   
1       0       0     0           0            0                0                 0        0        0           0     0    0       0        0     0              0        0     0         0                      0         0         0           0     0         0     0           0                    0               0      0     0    0          0    0             0                 5308   
                                                                                                                                                                                                                                                                                                                                                                        1                  196   
                                                                                                                                                                                                                                                                                                        1               0      0     0    0          0    1             0                  160   
                                                                                                                                                                                                                                                                                                                                                                        1                   17   

                                                                                                                                                                                                                                                                                                                                                                                       original  \
related request offer aid_related medical_help medical_products search_and_rescue security military child_alone water food shelter clothing money missing_people refugees death other_aid infrastructure_related transport buildings electricity tools hospitals shops aid_centers other_infrastructure weather_related floods storm fire earthquake cold other_weather direct_report             
0       0       0     0           0            0                0                 0        0        0           0     0    0       0        0     0              0        0     0         0                      0         0         0           0     0         0     0           0                    0               0      0     0    0          0    0             0                  3395   
1       0       0     0           0            0                0                 0        0        0           0     0    0       0        0     0              0        0     0         0                      0         0         0           0     0         0     0           0                    0               0      0     0    0          0    0             0                  1964   
                                                                                                                            

#### Euclidean Distance

In [116]:
categories_df = df[df.columns[3:]]

In [123]:
categories_df.T.dot(categories_df)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,20093,4474,118,10860,2084,1313,724,471,860,0,...,309,1151,7297,2155,2443,282,2455,530,1376,5075
request,4474,4474,0,4030,699,453,239,155,69,0,...,77,191,1181,353,286,39,439,88,201,3391
offer,118,0,118,107,12,21,3,0,4,0,...,1,0,22,9,4,1,5,0,4,36
aid_related,10860,4030,107,10860,2084,1313,724,471,860,0,...,290,825,4234,1487,1238,188,1320,367,933,4015
medical_help,2084,699,12,2084,2084,632,178,95,76,0,...,71,142,887,533,209,42,228,73,171,682
medical_products,1313,453,21,1313,632,1313,112,69,38,0,...,65,109,484,252,123,34,130,59,110,396
search_and_rescue,724,239,3,724,178,112,724,71,59,0,...,29,86,478,229,116,31,150,54,81,237
security,471,155,0,471,95,69,71,471,115,0,...,18,44,256,141,47,26,64,28,35,158
military,860,69,4,860,76,38,59,115,860,0,...,26,70,207,70,37,30,54,25,52,96
child_alone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
np.sqrt(categories_df.T.dot(categories_df == 0) + (categories_df == 0).T.dot(categories_df))

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,0.000000,124.975998,141.332940,96.088501,134.197615,137.040140,139.172555,140.078549,138.683092,141.749780,...,140.655608,137.629939,113.119406,133.932819,132.853303,140.751554,132.808132,139.867795,136.810087,122.547950
request,124.975998,0.000000,67.764297,85.287748,71.833140,69.864154,68.702256,68.080834,72.083285,66.887966,...,68.036755,72.408563,97.000000,76.961029,79.655508,68.395906,77.788174,69.483811,73.810568,52.602281
offer,141.332940,67.764297,0.000000,103.749699,46.669048,37.269290,28.913665,24.269322,31.144823,10.862780,...,20.615528,35.623026,85.854528,47.486840,50.527220,19.949937,50.626080,25.455844,38.548671,71.561163
aid_related,96.088501,85.287748,103.749699,0.000000,93.680307,97.708751,100.677704,101.926444,100.000000,104.211324,...,102.902867,101.788997,98.432718,100.204790,104.052871,103.759337,103.319892,103.227903,101.833197,88.910067
medical_help,134.197615,71.833140,46.669048,93.680307,0.000000,46.184413,49.517674,48.631266,52.839379,45.650849,...,47.444705,54.323107,87.218117,56.329388,64.101482,47.770284,63.898357,49.678969,55.839054,76.124897
medical_products,137.040140,69.864154,37.269290,97.708751,46.184413,0.000000,42.579338,40.570926,45.793013,36.235342,...,38.626416,47.391982,87.418534,54.442630,59.245253,39.076847,59.228372,41.533119,49.689033,74.806417
search_and_rescue,139.172555,68.702256,28.913665,100.677704,49.517674,42.579338,0.000000,32.449961,38.288379,26.907248,...,31.224990,41.267421,84.053554,49.203658,54.175640,30.724583,53.656314,33.852622,44.022721,72.972598
security,140.078549,68.080834,24.269322,101.926444,48.631266,40.570926,32.449961,0.000000,33.181320,21.702534,...,27.276363,39.166312,85.182158,48.414874,53.103672,26.476405,52.896125,30.740852,42.154478,72.318739
military,138.683092,72.083285,31.144823,100.000000,52.839379,45.793013,38.288379,33.181320,0.000000,29.325757,...,33.421550,43.255058,87.994318,53.619026,56.824291,32.893768,56.630381,36.606010,46.173586,75.782584
child_alone,141.749780,66.887966,10.862780,104.211324,45.650849,36.235342,26.907248,21.702534,29.325757,0.000000,...,17.578396,33.926391,85.422479,46.421978,49.426713,16.792856,49.547957,23.021729,37.094474,71.239034


In [126]:
categories_df.corr().values.tolist()

[[1.0,
  0.250384256226178,
  0.03711443293532311,
  0.4641869665032515,
  0.16220304168586816,
  0.12673972677133852,
  0.09301932056370889,
  0.0746568519042929,
  0.10165173140553546,
  nan,
  0.14406283543963033,
  0.19552808085568787,
  0.1717400260594787,
  0.06914021416015458,
  0.08476225024139751,
  0.05918517608320521,
  0.10256474116853309,
  0.12057223417098814,
  0.21472568299691502,
  0.14557546747018918,
  0.12094207340246162,
  0.12775266151823342,
  0.07943834598140634,
  0.04311633335629506,
  0.0576597005780392,
  0.037429074765333276,
  0.0602804157605238,
  0.11827962321433574,
  0.34279528426729167,
  0.1651861567036205,
  0.17694031866685375,
  0.0575566283702655,
  0.1774191389052958,
  0.07928579802660463,
  0.1299091386411773,
  0.2704360609322431],
 [0.250384256226178,
  1.0,
  -0.030505085867698487,
  0.44801091461458925,
  0.12868446835174577,
  0.10640810603141748,
  0.07142210690531169,
  0.05695507232671091,
  -0.04427312314407363,
  nan,
  0.23926781697

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [74]:
engine = create_engine('sqlite:///disaster_response_data.db')
df.to_sql('messages_categorized', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.